In [21]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv(r'C:\Users\One\Git\COURSERA YANDEX\Course 4\week 17\gene_high_throughput_sequencing.csv')

In [9]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Columns: 15750 entries, Patient_id to EIF1AY
dtypes: float64(15748), object(2)
memory usage: 8.7+ MB


In [11]:
from scipy import stats

In [12]:
from scipy.stats import ttest_ind

In [13]:
p_values = pd.DataFrame(index = [0, 1], columns = data.columns[2:])

for column in data.columns[2:]:
    p_values[column][0] = (stats.ttest_ind(data[data['Diagnosis'] == 'normal'][column], data[data['Diagnosis'] == 'early neoplasia'][column], equal_var=False).pvalue)
    p_values[column][1] = (stats.ttest_ind(data[data['Diagnosis'] == 'early neoplasia'][column], data[data['Diagnosis'] == 'cancer'][column], equal_var=False).pvalue)

In [14]:
p_values.head()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,0.690766,3.17853e-05,0.0602727,0.826429,0.0498762,0.144267,0.329108,0.0237124,0.240148,0.0379328,...,0.463274,0.806008,0.424543,0.740591,0.414922,0.640904,0.830134,0.670395,0.793925,0.661031
1,0.413735,0.653429,0.0795556,0.287581,0.463292,0.00768133,0.481306,0.57883,0.00074024,0.712687,...,0.107366,0.458907,0.893433,0.467608,0.881584,0.659369,0.330617,0.542939,0.565753,0.63901


In [15]:
print((p_values.iloc[0] < 0.05).sum(), "answer1_1.txt")

1575 answer1_1.txt


In [16]:
print((p_values.iloc[1] < 0.05).sum(), "answer1_1.txt")

3490 answer1_1.txt


In [17]:
import statsmodels.stats.multitest as smm

In [18]:
reject_0, p_corrected_0 = smm.multipletests(p_values.iloc[0],alpha=0.025, method = 'h')[:2]
reject_1, p_corrected_1 = smm.multipletests(p_values.iloc[1],alpha=0.025, method = 'h')[:2]

In [19]:
""""weak_ind_0 = np.where(p_corrected_0 < 0.05)
weak_ind_1 = np.where(p_corrected_1 < 0.05)"""

'"weak_ind_0 = np.where(p_corrected_0 < 0.05)\nweak_ind_1 = np.where(p_corrected_1 < 0.05)'

In [22]:
p_corr = np.array([p_corrected_0, p_corrected_1])
_, p_corr_bonf, _, _ = smm.multipletests(p_corr, is_sorted=True, method='bonferroni')

p_corr_bonf_normal_neoplasia = p_corr_bonf[0][np.where(p_corr_bonf[0] < 0.05)].shape[0]
p_corr_bonf_neoplasia_cancer = p_corr_bonf[1][np.where(p_corr_bonf[1] < 0.05)].shape[0]

In [24]:
c_0 = data[data['Diagnosis'] == 'normal'].iloc[:, 2:].iloc[:, np.where(p_corr_bonf[0] < 0.05)[0]].mean()
t_0 = data[data['Diagnosis'] == 'early neoplasia'].iloc[:, 2:].iloc[:, np.where(p_corr_bonf[0] < 0.05)[0]].mean()

In [25]:
def fold_change(C, T, limit=1.5):
    '''
    C - control sample
    T - treatment sample
    '''
    if T >= C:
        fc_stat = T / C
    else:
        fc_stat = -C / T

    return (np.abs(fc_stat) > limit), fc_stat

In [26]:
accept_0 = 0
for C, T in zip(c_0, t_0):
    if fold_change(C,T)[0] == True and (abs(fold_change(C,T)[1]) > 1.5):
        accept_0 += 1

In [35]:
c_1 = data[data['Diagnosis'] == 'early neoplasia'].iloc[:, np.where(p_corr_bonf[0] < 0.05)[0]].mean()
t_1 = data[data['Diagnosis'] == 'cancer'].iloc[:, np.where(p_corr_bonf[1] < 0.05)[0]].mean()

In [36]:
accept_1 = 0
for C, T in zip(c_1, t_1):
    if (fold_change(C,T)[0] == True) and (abs(fold_change(C,T)[1]) > 1.5):
        accept_1+= 1

In [43]:
reject_0, p_corrected_0 = smm.multipletests(p_values.iloc[0],alpha=0.025, method = 'fdr_bh')[:2]
reject_1, p_corrected_1 = smm.multipletests(p_values.iloc[1],alpha=0.025, method = 'fdr_bh')[:2]

p_corr = np.array([p_corrected_0, p_corrected_1])
_, p_corr_bonf, _, _ = smm.multipletests(p_corr, is_sorted=True, method='bonferroni')

p_corr_bonf_normal_neoplasia = p_corr_bonf[0][np.where(p_corr_bonf[0] < 0.05)].shape[0]
p_corr_bonf_neoplasia_cancer = p_corr_bonf[1][np.where(p_corr_bonf[1] < 0.05)].shape[0]

c_0 = data[data['Diagnosis'] == 'normal'].iloc[:, 2:].iloc[:, np.where(p_corr_bonf[0] < 0.05)[0]].mean()
t_0 = data[data['Diagnosis'] == 'early neoplasia'].iloc[:, 2:].iloc[:, np.where(p_corr_bonf[0] < 0.05)[0]].mean()

accept_3 = 0
for C, T in zip(c_0, t_0):
    if fold_change(C,T)[0] == True and (abs(fold_change(C,T)[1]) > 1.5):
        accept_3 += 1

c_1 = data[data['Diagnosis'] == 'early neoplasia'].iloc[:, np.where(p_corr_bonf[1] < 0.05)[0]].mean()
t_1 = data[data['Diagnosis'] == 'cancer'].iloc[:, np.where(p_corr_bonf[1] < 0.05)[0]].mean()

accept_4 = 0
for C, T in zip(c_1, t_1):
    if (fold_change(C,T)[0] == True) and (abs(fold_change(C,T)[1]) > 1.5):
        accept_4+= 1

In [49]:
accept_4

"не стал разбираться почему, но 6й ответ на 1 больше"

76